In [1]:
!pip install -q transformers accelerate bitsandbytes sentencepiece tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.3 MB/s eta 0:00:00


Load the model:

In [2]:
from transformers import pipeline

classifier = pipeline("text-generation", model="0x404/ccs-code-llama-7b", device_map="auto")
tokenizer = classifier.tokenizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

Device set to use cuda:0


Define prompt and classification functions:

In [ ]:
def prepare_prompt(commit_message, git_diff, context_window=1024):
  prompt_head = """<s>[INST] <<SYS>>
You are a commit classifier based on commit message and code diff. Please classify the given commit into one of the ten categories: docs, perf, style, refactor, feat, fix, test, ci, build, and chore. The definitions are:
**feat**: new features. **fix**: bug fixes. **perf**: performance improvements. **style**: code formatting/linting. **refactor**: restructure without behavior change. **docs**: documentation. **test**: tests. **ci**: CI/CD changes. **build**: build system. **chore**: other changes.
<</SYS>>

"""
  prompt_head_encoded = tokenizer.encode(prompt_head, add_special_tokens=False)

  prompt_message = f"- given commit message:\n{commit_message}\n"
  prompt_message_encoded = tokenizer.encode(
        prompt_message, max_length=64, truncation=True, add_special_tokens=False
    )

  prompt_diff = f"- given commit diff: \n{git_diff}\n"
  remaining_length = (
      context_window - len(prompt_head_encoded) - len(prompt_message_encoded) - 6
  )
  prompt_diff_encoded = tokenizer.encode(
      prompt_diff,
      max_length=remaining_length,
      truncation=True,
      add_special_tokens=False,
  )

  prompt_end = tokenizer.encode(" [/INST]", add_special_tokens=False)
  return tokenizer.decode(
      prompt_head_encoded + prompt_message_encoded + prompt_diff_encoded + prompt_end
  )

def classify_commit(commit_message, git_diff):
  prompt = prepare_prompt(commit_message, git_diff)
  result = classifier(prompt, max_new_tokens=10, pad_token_id=tokenizer.eos_token_id)
  classification = result[0]["generated_text"].split()[-1].strip()
  return classification

Import json file with commits:

In [4]:
from google.colab import files

uploaded = files.upload()

In [5]:
input_path = "/content/commits.json"
output_path = "/content/classified_commits.json"

Classify the commits:

In [ ]:
import json
from tqdm import tqdm

with open(input_path, "r", encoding="utf-8") as f:
  commits = json.load(f)

classified_commits = []

for idx, commit in enumerate(tqdm(commits, desc="Classifying")):
  try:
    message = commit["message"]
    diff = commit["diff"]
    label = classify_commit(message, diff)
    commit["predicted_label"] = label
    classified_commits.append(commit)

    # Save every 100 commits
    if idx % 100 == 0:
        with open(output_path, "w", encoding="utf-8") as out:
            json.dump(classified_commits, out, ensure_ascii=False, indent=2)

  except Exception as e:
    print(f"Error at commit {idx}: {e}")
    continue

# Final save
with open(output_path, "w", encoding="utf-8") as out:
  json.dump(classified_commits, out, ensure_ascii=False, indent=2)

print(f"Done! Saved to {output_path}")


Classifying:   4%|▍         | 132/3411 [14:59<6:48:49,  7.48s/it]

Download the classification:

In [ ]:
from google.colab import files
files.download(output_path)